In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Spotify 2024–2025 Hit Song Prediction

Our goal in this project is to use audio characteristics like tempo, danceability, energy, and more to determine whether a song will become a **hit**.


We create a classification model that calculates the **probability of a song becoming a hit** using machine learning techniques.

## Load the Dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('/kaggle/input/top-spotify-songs/universal_top_spotify_songs.csv')
data.head()


## 📊 Overview of the Dataset

The dataset has Spotify's top songs from 2024, along with different features for each song.


We'll look at the structure and see if there is any missing or bad data.




In [ ]:
# Display basic information
print(data.info())

# Display summary statistics
print(data.describe())

# Check for missing values
print(data.isnull().sum())


## Preprocessing & Data Cleaning

Let's get the data clean and ready for modeling.

## Exploratory Data Analysis (EDA)

We visualize feature distributions and explore patterns in the dataset.



In [ ]:
# Visualize distributions
numeric_features = ['tempo', 'danceability', 'popularity', 'loudness', 'speechiness','acousticness', 'valence', 'liveness', 'daily_movement']  # Columns to analyze
print(data.columns)
print(data[numeric_features].dtypes)
print(data.shape)

In [ ]:
# Change infinite values to NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Check for missing values
print(data.isnull().sum())

sns.pairplot(data[numeric_features].sample(15000, random_state=42)) # Randomly selects 5000 sample rows
plt.show()

## Establishing the Target Variable

If a song's popularity score is greater than 90, we consider it a **"hit"**.

In [ ]:
# Define hit
data['hit'] = data['popularity'].apply(lambda x: 1 if x > 90 else 0)
x = data[numeric_features]
y = data['hit']

from sklearn.model_selection import train_test_split
# Split data into a training dataset and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state=1)

# Verification checks
print(f"Total number of samples: {len(data)}")
print(f"Training set size: {len(x_train)}")
print(f"Test set size: {len(x_test)}")
print(f"Hit rate (full data): {y.mean():.2%}")
print(f"Hit rate (training): {y_train.mean():.2%}")
print(f"Hit rate (test): {y_test.mean():.2%}")


## Model Building

We use Logistic Regression as our classifier.


In [ ]:
from sklearn.linear_model import LogisticRegression
# Create and train model
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

# Create results DataFrame 
results = pd.DataFrame({
    'name': data.loc[x_test.index, 'name'],  # Get names from original data
    'hit_probability': model.predict_proba(x_test)[:, 1],
    'actual_hit': y_test.values})  # Use values to avoid index issues

# Sort and display 
sorted_results = results.sort_values('hit_probability', ascending=False).reset_index(drop=True)
print("Top 20 predictions:")
print(sorted_results.head(20))

# Consistency check
is_consistent = True
for i in range(1, min(20, len(sorted_results))):  # Check only top 20
    prob1 = sorted_results.loc[i, 'hit_probability']
    prob0= sorted_results.loc[i-1, 'hit_probability']
    actual1 = sorted_results.loc[i, 'actual_hit']
    actual0 = sorted_results.loc[i-1, 'actual_hit']
    
    if (prob1 > prob0) and (actual1 < actual0):
        is_consistent = False
        print(f"Inconsistency found at row {i}")
        break

print("\nConsistency check:", "✅ Yes" if is_consistent else "❌ No")


## Predicting Manual User Input

We let users manually enter song values because Kaggle does not support the live Spotify API.

In [ ]:
print("\n Please enter the following music features:")

def float_input(prompt):
    while True:
        try:
            return float(input(prompt))
        except:
            print("Please enter a valid number.")

user_input = {'tempo': float_input("Tempo (e.g., 120.0): "),
    'danceability': float_input("Danceability (between 0 and 1): "),
    'popularity': float_input("Estimated popularity score (0-100): "),
    'loudness': float_input("Loudness (e.g., -5.0): "),
    'speechiness': float_input("Speechiness (between 0 and 1): "),
    'acousticness': float_input("Acousticness (between 0 and 1): "),
    'valence': float_input("Valence (between 0 and 1): "),
    'liveness': float_input("Liveness (between 0 and 1): "),
    'daily_movement': float_input("Daily Movement (e.g., 1.5): ")}

input_df = pd.DataFrame([user_input])



In [ ]:
# Make prediction
hit_prob = model.predict_proba(input_df)[:, 1][0]
hit_label = "✅ Might Be a Hit!" if hit_prob > 0.5 else "❌ Might Not Be a Hit"

# Display result
print(f"\n Predicted hit probability: {hit_prob * 100:.2f}%")
print(f" Prediction: {hit_label}")


## Findings and Insights

On test data, the model performs fairly well. Manual input is adaptable and enables user-driven prediction.

Features that affect hit potential include danceability and tempo.

### Future Developments: - Experiment with various models, such as Random Forest or XGBoost - Include artist popularity (if external APIs are permitted outside of Kaggle)